# Normal Distributions

In [ ]:
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np

### One Dimensional Normal PDF

In [ ]:
def normalpdf_1d(X,mean,var):
    return np.exp(-0.5*np.square(X-mean)/var)/np.sqrt(2*np.pi*var)

### Plot

In [ ]:
# create plot of 1d normal probability distribution function
npoint = 100
X = np.linspace(-5,5,npoint)
Z1 = normalpdf_1d(X,0,1)
Z2 = normalpdf_1d(X,0.5,4)
Z3 = normalpdf_1d(X,-1,0.25)

In [ ]:
# plot pdf
plt.figure()
plt.plot(X,Z1,"r-",label="$\mu=0$, $var=1$")
plt.plot(X,Z2,"b-",label="$\mu=0.5$, $var=4$")
plt.plot(X,Z3,"g-", label="$\mu=-1$, $var=0.25$")
plt.legend()

### Multi-dimensional Normal PDF

In [ ]:
def normalpdf(X,mean,Sigma):
    # X is 2d array with d rows and nsample columns
    # mean is 2d array d rows and 1 column
    # Sigma covariance matrix is 2d array with d rows and d columns 
    # Z ouptut (1 row and nsample columns) is normal pdf
    d,nsample = X.shape
    Z = np.zeros((1,nsample))
    invSigma = np.linalg.inv(Sigma)
    detSigma = np.linalg.det(Sigma)
    for i in range(nsample):
        Z[0,i] = np.exp(-0.5*np.dot((X[:,[i]]-mean).T,np.matmul(invSigma,X[:,[i]]-mean)))/np.sqrt(np.power(2*np.pi,d)*detSigma)
    return Z

### Surface Plot

In [ ]:
# create gridpoints in x0-x1 plane
npoint = 10
Cov = np.array([[1,-0.5],[-0.5,2]])
mean = np.array([[0.5],[0.5]])
x0max = mean[0,0]+3
x0min = mean[0,0]-3
x1max = mean[1,0]+3
x1min = mean[1,0]-3
X0val,X1val = np.meshgrid(np.linspace(x0min,x0max,npoint),np.linspace(x1min,x1max,npoint))

In [ ]:
plt.figure()
plt.scatter(X0val,X1val)

In [ ]:
# compute 2d normald pdf at gridpoints
# create dataset of X0 and X1 coordinates
X = np.concatenate((np.reshape(X0val,(1,npoint*npoint)),np.reshape(X1val,(1,npoint*npoint))),axis=0)
# compute normal pdf Z  row vector of dimension 1 x npoint*npoint
Z = normalpdf(X,mean,Cov)
# reshape Z into grid of npoint x npoint
Zval = np.reshape(Z,(npoint,npoint))

In [ ]:
fig = plt.figure()
ax = plt.axes(projection="3d")
ax.plot_surface(X0val,X1val,Zval,cmap=cm.jet)
ax.set_xlabel("X0")
ax.set_ylabel("X1")
ax.set_zlabel("Z")
ax.view_init(60, -30)

### Contours in x0-x1 plane

In [ ]:
# plot contours of normal pdf in 2d
fig,ax = plt.subplots()
ax.set_aspect("equal")
plt.contour(X0val,X1val,Zval)

### Create contour using Matplotlib Ellipse

In [ ]:
# Create contour for C = 0.02
U,Sigma,Vt = np.linalg.svd(Cov)
print("Sigma: {}".format(Sigma))
contour = 0.02
alpha=np.sqrt(-2*np.log(contour*2*np.pi*np.sqrt(np.linalg.det(Cov))))
fig,ax  = plt.subplots()
ax.set_aspect("equal")
width = 2*alpha*np.sqrt(Sigma[0])
height = 2*alpha*np.sqrt(Sigma[1])
ax.set_xlim(x0min,x0max)
ax.set_ylim(x1min,x1max)
# compute angle - convert from radians to degrees
angle = np.arctan(U[1][0]/(U[0,0]+1e-10))*180/np.pi
ellipse = Ellipse(xy=np.squeeze(mean),width=width,height=height,angle=angle, fill=False)
ax.add_patch(ellipse)